In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
feature_df = pd.read_csv('clean/MovieFeatureVector.csv')
feature_df.pop('tmdbId')
feature_df = feature_df.set_index('movieId')

In [ ]:
feature_df

<h2>Index in  movie feature vector</h2>
<ul>
<li>0:106 Languages</li>
<li>107 adult</li>
<li>108 voteaverage</li>
<li>109:111 VAD</li>
<li>112:411 Word2Vec</li>
</ul>

<p>Total dimension of movie feature vector 412</p>


    

In [ ]:
# print(feature_df.iloc[:,0:106])
# print(feature_df.iloc[:,106:107])
# print(feature_df.iloc[:,107:108])
# print(feature_df.iloc[:,108:111])
feature_df.iloc[:,0:411]


<h1>MOVIE EMBEDDINGS</h1>
<h2>MVAE</h2>


In [ ]:
movie_feature = feature_df.to_numpy()


In [ ]:
movie_feature.shape
INPUT_SIZE = movie_feature.shape[1]
INTERMEDIATE_DIM = 50

In [ ]:
import numpy as np
import os
import pandas as pd
import time

import tensorflow as tf
# import tensorflow_probability as tfp
# tfd = tfp.distributions
# tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
class MVAE(tfk.Model):
    
    def __init__(self, dim_z, kl_weight=1, name="autoencoder", **kwargs):
        super(MVAE, self).__init__(name=name, **kwargs)
        self.dim_x = INPUT_SIZE
        self.dim_z = dim_z
        self.kl_weight = kl_weight
        self.initializer = tfk.initializers.HeNormal()
        self.encoder = self.encoder_z()
        self.decoder = self.decoder_x()
    # Sequential API encoder
    def encoder_z(self):
        layers = [tfkl.InputLayer(input_shape=self.dim_x)]
        layers.append(tfkl.Dense(INTERMEDIATE_DIM,kernel_initializer=self.initializer, activation='relu'))
        # *2 because number of parameters for both mean and (raw) standard deviation
        layers.append(tfkl.Dense(self.dim_z*2,kernel_initializer=self.initializer,  activation=None))
        return tfk.Sequential(layers)
    
    def encode(self, x_input):
        mu, rho = tf.split(self.encoder(x_input), num_or_size_splits=2, axis=1)
        sd = tf.math.log(1+tf.math.exp(rho))
        z_sample = mu + sd * tf.random.normal(shape=(self.dim_z,))
        return z_sample, mu, sd
    
    # Sequential API decoder
    def decoder_x(self):
        layers = [tfkl.InputLayer(input_shape=self.dim_z)]
        layers.append(tfkl.Dense(INTERMEDIATE_DIM,kernel_initializer=self.initializer,  activation='relu'))
        layers.append(tfkl.Dense(self.dim_x,kernel_initializer=self.initializer, activation='sigmoid'))
  
        return tfk.Sequential(layers, name='decoder')
    
    def call(self, x_input):
        z_sample, mu, sd = self.encode(x_input)
        kl_divergence = tf.math.reduce_mean(- 0.5 * 
                tf.math.reduce_sum(1+tf.math.log(
                tf.math.square(sd))-tf.math.square(mu)-tf.math.square(sd), axis=1))
        x_logits = self.decoder(z_sample)
        # VAE_MNIST is inherited from tfk.Model, thus have class method add_loss()
        self.add_loss(self.kl_weight * kl_divergence)
        return x_logits
    
# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1])
    return tf.math.reduce_mean(neg_log_likelihood)


In [ ]:

####################   The following code shows how to train the model   ####################
# set hyperparameters
epochs = 10
batch_size = 128
lr = 0.0001
latent_dim=3
kl_w=3
vae = VAE_MNIST(dim_z=latent_dim, kl_weight=kl_w)
vae.encoder.summary()
vae.decoder.summary()

In [ ]:

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)


####################   The following code shows how to train the model   ####################
# set hyperparameters
epochs = 10
batch_size = 128
lr = 0.0001
latent_dim=3
kl_w=3
# compile and train tfk.Model
vae = MVAE(dim_z=latent_dim, kl_weight=kl_w)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
            loss=custom_sigmoid_cross_entropy_loss_with_logits, metrics=[tfk.metrics.Recall(top_k=50)])
train_history = vae.fit(x=movie_feature, y=movie_feature, batch_size=batch_size, epochs=epochs, 
                        verbose=1,  shuffle=True, callbacks=[cp_callback])

In [ ]:
!mkdir -p saved_model
vae.save_weights('saved_model/my_model3.h5')

In [ ]:
vae.weights

In [ ]:
new_vae = MVAE(dim_z=latent_dim, kl_weight=kl_w)
new_vae.built=True
new_var = tf.keras.models.load_model('saved_model/my_model')

new_vae.weights

<h2>Keras Docs MOd</h2>

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [ ]:
# Movies
# ORIGINAL DIM/ INPUT SIZE
INPUT_SIZE = movie_feature.shape[1]
# print(INPUT_SIZE)
INTERMEDIATE_DIM = 50
LATENT_DIM = 3
BATCH_SIZE = 128



encoder_inputs = keras.Input(shape=(INPUT_SIZE))
x = layers.Dense(INTERMEDIATE_DIM, activation="relu")(encoder_inputs)
z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
# encoder.summary()

latent_inputs = keras.Input(shape=(LATENT_DIM,))
x = layers.Dense(INTERMEDIATE_DIM,activation='relu')(latent_inputs)
decoder_outputs = layers.Dense(INPUT_SIZE, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
# decoder.summary()


activation

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
  
    def normal_log_pdf(self, sample, mean, var, raxis=1):
        logvar = tf.math.log(var)
        log2pi = tf.math.log(2. * np.pi)
        return tf.reduce_sum(-.5 * ((sample-mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

    @tf.function
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            # total_loss = self.vae_cost(data)
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *=INPUT_SIZE
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }


In [ ]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

STEPS_PER_EPOCH = movie_feature.shape[0] 
SAVE_PERIOD = 5

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1, save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))




vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adagrad(1e-6))
vae.encoder.summary()
vae.decoder.summary()


In [ ]:
EPOCHS = 50
history = vae.fit(movie_feature, epochs=EPOCHS, batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

loss_train = history.history['loss']
# loss_kl = history.history['kl_loss']
epochs = range(EPOCHS)
plt.plot(epochs, loss_train, 'g', label='Reconstruction loss')
# plt.plot(epochs, loss_kl, 'b', label='KL Loss' )
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
history.history

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
train_images = np.expand_dims(x_train, -1).astype("float32") / 255
test_images = np.expand_dims(x_test, -1).astype("float32") / 255

In [ ]:
test_images.shape

In [ ]:
class VAE_MNIST(tfk.Model):
    
    def __init__(self, dim_z, kl_weight=1, name="autoencoder", **kwargs):
        super(VAE_MNIST, self).__init__(name=name, **kwargs)
        self.dim_x = (28, 28, 1)
        self.dim_z = dim_z
        self.encoder = self.encoder_z()
        self.decoder = self.decoder_x()
        self.kl_weight = kl_weight
        
    # Sequential API encoder
    def encoder_z(self):
        layers = [tfkl.InputLayer(input_shape=self.dim_x)]
        layers.append(tfkl.Conv2D(filters=32, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        layers.append(tfkl.Conv2D(filters=64, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        layers.append(tfkl.Flatten())
        # *2 because number of parameters for both mean and (raw) standard deviation
        layers.append(tfkl.Dense(self.dim_z*2, activation=None))
        return tfk.Sequential(layers)
    
    def encode(self, x_input):
        mu, rho = tf.split(self.encoder(x_input), num_or_size_splits=2, axis=1)
        sd = tf.math.log(1+tf.math.exp(rho))
        z_sample = mu + sd * tf.random.normal(shape=(self.dim_z,))
        return z_sample, mu, sd
    
    # Sequential API decoder
    def decoder_x(self):
        layers = [tfkl.InputLayer(input_shape=self.dim_z)]
        layers.append(tfkl.Dense(7*7*32, activation=None))
        layers.append(tfkl.Reshape((7,7,32)))
        layers.append(tfkl.Conv2DTranspose(filters=64, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        layers.append(tfkl.Conv2DTranspose(filters=32, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        layers.append(tfkl.Conv2DTranspose(filters=1, kernel_size=3, strides=1, 
                                           padding='same'))
        return tfk.Sequential(layers, name='decoder')
    
    def call(self, x_input):
        z_sample, mu, sd = self.encode(x_input)
        kl_divergence = tf.math.reduce_mean(- 0.5 * 
                tf.math.reduce_sum(1+tf.math.log(
                tf.math.square(sd))-tf.math.square(mu)-tf.math.square(sd), axis=1))
        x_logits = self.decoder(z_sample)
        # VAE_MNIST is inherited from tfk.Model, thus have class method add_loss()
        self.add_loss(self.kl_weight * kl_divergence)
        return x_logits
    
# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1, 2, 3])
    return tf.math.reduce_mean(neg_log_likelihood)

  
####################   The following code shows how to train the model   ####################
# set hyperparameters
epochs = 10
batch_size = 128
lr = 0.0001
latent_dim=16
kl_w=3
# compile and train tfk.Model
vae = VAE_MNIST(dim_z=latent_dim, kl_weight=kl_w)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
            loss=custom_sigmoid_cross_entropy_loss_with_logits)
train_history = vae.fit(x=train_images, y=train_images, batch_size=batch_size, epochs=epochs, 
                        verbose=1, validation_data=(test_images, test_images), shuffle=True)

In [ ]:
train_history.history

In [ ]:
import matplotlib.pyplot as plt

loss_train = train_history.history['loss']
# loss_kl = history.history['kl_loss']
epochs = range(10)
plt.plot(epochs, loss_train, 'g', label='Reconstruction loss')
# plt.plot(epochs, loss_kl, 'b', label='KL Loss' )
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
keras = tf.keras

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed
    
# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1])
    return tf.math.reduce_mean(neg_log_likelihood)


In [ ]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

In [ ]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

In [ ]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

<h2>Better I guess</h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
keras = tf.keras

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding movie."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps movie vector to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded movie vector, back into a movie feature vector."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = tf.reduce_mean(-0.5 * tf.reduce_sum(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        ,axis=1))
        self.add_loss(kl_loss)
        return reconstructed
    

# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1])
    return tf.math.reduce_mean(neg_log_likelihood)


In [ ]:
train_size=60000
validation_size=10000
np.random.shuffle(movie_feature)
train_dataset= movie_feature[:train_size]
validation_dataset =  movie_feature[train_size:train_size+validation_size]
print(train_dataset.shape)
validation_dataset.shape

In [ ]:
INTERMEDIATE_DIM = 50
LATENT_DIM = 3
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

vae = VariationalAutoEncoder(INPUT_SIZE, INTERMEDIATE_DIM, LATENT_DIM, name='MVAE')


optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

vae.compile(optimizer, loss=custom_sigmoid_cross_entropy_loss_with_logits)
train_history = vae.fit(train_dataset, train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_data=(validation_dataset, validation_dataset))

In [ ]:
import matplotlib.pyplot as plt
loss_train = train_history.history['loss']
loss_val = train_history.history['val_loss']
epochs = range(EPOCHS)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# for i in train_dataset:
#     print(i)

In [ ]:
_,_,predicted= vae.encoder(movie_feature)

In [ ]:
predicted_np = predicted.numpy()
np.save('tmp/embed_movie',predicted_np)

In [ ]:
predicted_np_load = np.load('tmp/embed_movie.npy')
predicted_np_load

In [ ]:
_,_,p = vae.encoder(movie_feature[2].reshape(1,411))

In [ ]:
p

<h2>HVAE Hybrid Variational Autoencoder</h2>

In [4]:
import numpy as np
from scipy import sparse
def save_sparse_matrix(filename, x):
    x_coo = x.tocoo()
    row = x_coo.row
    col = x_coo.col
    data = x_coo.data
    shape = x_coo.shape
    np.savez(filename, row=row, col=col, data=data, shape=shape)

def load_sparse_matrix(filename):
    y = np.load(filename)
    z = sparse.coo_matrix((y['data'], (y['row'], y['col'])), shape=y['shape'])
    return z

In [ ]:
import pandas as pd
ratings_df = pd.read_csv('clean/rating_updated_clean.csv')

In [ ]:
movies_df = pd.read_csv('clean/movies_clean.csv')
movies_df.iloc[0].movieId

In [ ]:
from scipy.sparse import lil_matrix
from tqdm import tqdm
user_movie = lil_matrix((ratings_df['userId'].unique().shape[0]+1, movies_df.shape[0]))
for i in tqdm(range(0,movies_df.shape[0])):
  movies = ratings_df[ratings_df.movieId == movies_df.iloc[i]['movieId']]
  userIdList = movies['userId'].values
  for j in userIdList:
        user_movie[j,i] = 1

In [ ]:
save_sparse_matrix('tmp/dataset_matrix',user_movie)

In [ ]:
z = load_sparse_matrix('tmp/dataset_matrix.npz').tolil()

In [ ]:
ratings_df

In [5]:
x_train = load_sparse_matrix('tmp/dataset_matrix.npz')

In [6]:
# Convert to CSR format from stored COO format remove initial empty 
x_train = x_train.tocsr()
x_train = x_train[1:]

In [ ]:
x_train.shape

In [ ]:
embed_movie_feature = np.load('tmp/embed_movie.npy')
embed_movie_feature

In [ ]:
def shuffle_sparse_matrix(sparse_matrix):
    index = np.arange(np.shape(sparse_matrix)[0])
    np.random.shuffle(index)
    return sparse_matrix[index, :]

def batch_sparse_matrix(sparse_matrix, batch_size=64, shuffle=True):
    sparse_matrix = shuffle_sparse_matrix(sparse_matrix) if shuffle else sparse_matrix
    # Check if shuflle works
    # print(sparse_matrix[1].sum())
    index = np.arange(np.shape(sparse_matrix)[0])
    steps_per_epoch = int(index.shape[0]/batch_size)
    
    
    for i in range(0,steps_per_epoch):
        yield i+1, sparse_matrix[batch_size*i: batch_size*(i+1)]
    
    yield i+2,sparse_matrix[batch_size*(i+1): ]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
keras = tf.keras

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding movie."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(keras.Model):
    """Maps movie vector to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, vocab_size=1000, embed_dim=3, seq_length=1000, weights=[],name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="tanh")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.embedding_layer = layers.Embedding(vocab_size,embed_dim ,weights=weights, input_length=seq_length, trainable=True)
        self.flatten_layer = layers.Flatten()
        self.sampling = Sampling()

    def call(self, inputs):
        embed = self.embedding_layer(inputs)
        flat_embed = self.flatten_layer(embed)
        x = self.dense_proj(flat_embed)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(keras.Model):
    """Converts z, the encoded movie vector, back into a movie feature vector."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="tanh")
        self.dense_output = layers.Dense(original_dim, activation="softmax")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        vocab_size=1000,
        embed_dim=3,
        weights=[],
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim, vocab_size=vocab_size, embed_dim=embed_dim, seq_length=vocab_size, weights=weights)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = tf.reduce_mean(-0.5 * tf.reduce_sum(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        ,axis=1))
        self.add_loss(kl_loss)
        return reconstructed
    

# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1])
    return tf.math.reduce_mean(neg_log_likelihood)


In [ ]:
INPUT_DIM = x_train.shape[1] 
INTERMEDIATE_DIM = 600
LATENT_DIM = 200
EPOCHS = 15
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
VOCAB_SIZE = x_train.shape[1]
EMBED_DIM = 3

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# train_history = vae.fit(train_dataset, train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_data=(validation_dataset, validation_dataset))


In [ ]:
step, x_batch_train = next(train_dataset)
np.array(x_batch_train.todense()).shape

In [ ]:
train_dataset = batch_sparse_matrix(x_train, 128, False)
tf.keras.backend.set_floatx('float64')
vae = VariationalAutoEncoder( original_dim=INPUT_DIM,
        intermediate_dim=INTERMEDIATE_DIM,
        latent_dim=LATENT_DIM,
        vocab_size=VOCAB_SIZE,
        embed_dim=EMBEDINPUT_DIM = x_train.shape[1] 
INTERMEDIATE_DIM = 600
LATENT_DIM = 200
EPOCHS = 15
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
VOCAB_SIZE = x_train.shape[1]
EMBED_DIM = 3

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# train_history = vae.fit(train_dataset, train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_data=(validation_dataset, validation_dataset))
_DIM,
        weights=[embed_movie_feature])

@tf.function
def train_step(model, dataset,optimizer, epochs):
    # Iterate over epochs.
    for epoch in range(epochs):
        print("Start of epoch %d" % (epoch,))
      

        # Iterate over the batches of the dataset.
        for step, x_batch_train in train_dataset:
            x_batch_train = np.array(x_batch_train.todense())
      
            with tf.GradientTape() as tape:
                reconstructed = model(x_batch_train)
                # Compute reconstruction loss
                loss = custom_sigmoid_cross_entropy_loss_with_logits(x_batch_train, reconstructed)
                loss += sum(vae.losses)  # Add KLD regularization loss

            grads = tape.gradient(loss, vae.trainable_weights)
            optimizer.apply_gradients(zip(grads, vae.trainable_weights))


            if step % 1 == 0:
                print("step %d: mean loss = %.4f" % (step, loss.eval()))
                
train_step(vae, train_dataset, optimizer, EPOCHS)

In [ ]:
# from tqdm import tqdm
# tqdm.pandas()
# def func(row):
#     return row

# new_df = ratings_df.iloc[0:1000000].progress_apply(func,axis=1)

In [ ]:
userIdList = ratings_df['userId'].unique()
userIdList.shape[0]

In [ ]:
user_movie = np.zeros(())

In [ ]:
# ratings_df.to_csv('clean/rating_updated_clean.csv')

In [ ]:
user_movie.shape

In [ ]:
user_movies_dict = {}

for i in tqdm(userIdList[0:10]):
    movie_dict = {}
    for j in ratings_df[ratings_df['userId']==i]['movieId'].tolist():
        movie_dict[j] = 1
    user_movies_dict[i]= movie_dict
#     (ratings_df[ratings_df['userId']==i])

In [ ]:
len(user_movies_dict[1])

In [ ]:
len(user_movies_dict[4])

In [ ]:
from scipy.sparse import csr_matrix
a = csr_matrix([0,0,0,1])
a[0,1]=2

In [ ]:
a.todense()
print(ratings_df['userId'].unique().shape[0])
movies_df.shape

In [ ]:
z[3]

In [ ]:
import tensorflow as tf
sparse = tf.sparse.SparseTensor(indices=[[0,0]], values=[1],dense_shape=[1000000,1000000])

In [ ]:
sparse[0]

In [ ]:

movie_indices = np.array([range(1,26622)])
movie_indices = np.repeat(movie_indices, 64, axis = 0)

In [ ]:
movie_indices.shape

In [ ]:
sparse= csr_matrix(([1], ([0], [0])), shape=(25000000, 62000))

In [ ]:
sp = sparse.tolil()

In [ ]:
sp[0,2] = 3

In [ ]:
print(sp)

In [ ]:
sparse[0]=sp.tocsr()

In [ ]:
sp

In [ ]:
from scipy.sparse import lil_matrix

In [ ]:
lil= lil_matrix((100000, 62000))

In [ ]:
lil.shape

In [ ]:
lil[0,2]=3
lil[0,4]=1
lil[0,5]=2


In [ ]:
lil

In [33]:

movie_indices = np.array([range(0,x_train.shape[1])])
movie_indices = np.repeat(movie_indices, 128, axis = 0)
def nn_batch_generator(x, y, batch_size, samples_per_epoch):
    
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    x =  x[shuffle_index, :]
    y =  y[shuffle_index, :]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        x_batch = np.array(x[index_batch,:].todense()).astype(float)
        x_new_batch = x_batch*movie_indices
        
        counter += 1
        yield ([x_new_batch, x_batch], x_batch)
        if (counter >= number_of_batches):
            counter=0
            


In [34]:
generator =nn_batch_generator(x_train, x_train, 128, x_train.shape[0])

In [ ]:
from tqdm import tqdm
for i,j in tqdm(generator, total=int(x_train.shape[0]/128)):
    continue

 10%|█         | 130/1269 [01:57<13:25,  1.41it/s]

In [23]:
for i,j in generator:
    print(len(j))

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


KeyboardInterrupt: 

<h1>MOVIE EMBEDDINGS</h1>
<h2>MVAE</h2>


In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [3]:
feature_df = pd.read_csv('clean/MovieFeatureVector.csv')
feature_df.pop('tmdbId')
feature_df = feature_df.set_index('movieId')

In [4]:
feature_df

,ab,af,ak,am,ar,as,ay,az,bg,bm,...,290,291,292,293,294,295,296,297,298,299
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.026189,0.076369,-0.112352,0.019649,-0.069810,-0.107645,-0.013949,-0.042618,0.039779,-0.029764
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.116423,0.049320,-0.130721,-0.000997,-0.040562,-0.027616,0.000512,0.010408,-0.008199,-0.025869
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.018968,-0.003209,-0.119756,0.024853,0.037863,-0.056118,-0.038796,-0.042909,0.031052,0.032308
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000760,0.035324,-0.116697,-0.012555,-0.063151,-0.059833,-0.017836,-0.041529,-0.017190,-0.033720
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.108175,-0.023547,-0.139049,-0.065952,0.046830,0.004845,-0.008362,-0.027590,0.051061,0.021861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.055698,0.041496,-0.129931,-0.003363,-0.061127,0.003019,-0.000026,-0.056713,0.052466,-0.013415
209159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.047187,0.026757,-0.096046,0.005746,-0.052543,0.011463,0.032719,0.012167,0.093388,0.015554
209163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.081694,0.056029,-0.096148,-0.013727,-0.075042,-0.067631,-0.006398,-0.015436,0.072134,-0.044202


<h2>Index in  movie feature vector</h2>
<ul>
<li>0:106 Languages</li>
<li>107 adult</li>
<li>108 voteaverage</li>
<li>109:111 VAD</li>
<li>112:411 Word2Vec</li>
</ul>

<p>Total dimension of movie feature vector 412</p>


    

In [5]:
# print(feature_df.iloc[:,0:106])
# print(feature_df.iloc[:,106:107])
# print(feature_df.iloc[:,107:108])
# print(feature_df.iloc[:,108:111])
feature_df.iloc[:,0:411]


,ab,af,ak,am,ar,as,ay,az,bg,bm,...,290,291,292,293,294,295,296,297,298,299
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.026189,0.076369,-0.112352,0.019649,-0.069810,-0.107645,-0.013949,-0.042618,0.039779,-0.029764
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.116423,0.049320,-0.130721,-0.000997,-0.040562,-0.027616,0.000512,0.010408,-0.008199,-0.025869
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.018968,-0.003209,-0.119756,0.024853,0.037863,-0.056118,-0.038796,-0.042909,0.031052,0.032308
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000760,0.035324,-0.116697,-0.012555,-0.063151,-0.059833,-0.017836,-0.041529,-0.017190,-0.033720
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.108175,-0.023547,-0.139049,-0.065952,0.046830,0.004845,-0.008362,-0.027590,0.051061,0.021861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.055698,0.041496,-0.129931,-0.003363,-0.061127,0.003019,-0.000026,-0.056713,0.052466,-0.013415
209159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.047187,0.026757,-0.096046,0.005746,-0.052543,0.011463,0.032719,0.012167,0.093388,0.015554
209163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.081694,0.056029,-0.096148,-0.013727,-0.075042,-0.067631,-0.006398,-0.015436,0.072134,-0.044202


In [6]:
movie_feature = feature_df.to_numpy()


In [7]:
movie_feature.shape
INPUT_SIZE = movie_feature.shape[1]
INTERMEDIATE_DIM = 50

<h2>Better I guess</h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
keras = tf.keras

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding movie."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps movie vector to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded movie vector, back into a movie feature vector."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = tf.reduce_mean(-0.5 * tf.reduce_sum(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        ,axis=1))
        self.add_loss(kl_loss)
        return reconstructed
    

# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1])
    return tf.math.reduce_mean(neg_log_likelihood)


In [ ]:
train_size=60000
validation_size=10000
np.random.shuffle(movie_feature)
train_dataset= movie_feature[:train_size]
validation_dataset =  movie_feature[train_size:train_size+validation_size]
print(train_dataset.shape)
validation_dataset.shape

In [ ]:
INTERMEDIATE_DIM = 50
LATENT_DIM = 3
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

vae = VariationalAutoEncoder(INPUT_SIZE, INTERMEDIATE_DIM, LATENT_DIM, name='MVAE')


optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

vae.compile(optimizer, loss=custom_sigmoid_cross_entropy_loss_with_logits)
train_history = vae.fit(train_dataset, train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_data=(validation_dataset, validation_dataset))

In [ ]:
import matplotlib.pyplot as plt
loss_train = train_history.history['loss']
loss_val = train_history.history['val_loss']
epochs = range(EPOCHS)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# for i in train_dataset:
#     print(i)

In [ ]:
_,_,predicted= vae.encoder(movie_feature)

In [ ]:
predicted_np = predicted.numpy()
np.save('tmp/embed_movie',predicted_np)

In [ ]:
predicted_np_load = np.load('tmp/embed_movie.npy')
predicted_np_load

In [ ]:
_,_,p = vae.encoder(movie_feature[2].reshape(1,411))

In [ ]:
p